# Módulo 3: Gerando Insights com Análise e Visualização 📊

## Cenário

No Módulo 2, fizemos o trabalho pesado de limpar e organizar nossos dados de vendas. Agora, com um conjunto de dados limpo e confiável, podemos começar a parte mais divertida: **responder perguntas de negócio**.

O gerente da lanchonete quer saber:
1.  Qual produto gera mais receita?
2.  Em qual dia da semana vendemos mais?

Nossa missão é usar o poder do Pandas e das bibliotecas de visualização para transformar nossos dados em respostas claras e visuais.

### Passo 0: Configuração do Ambiente

Além do `pandas`, agora vamos importar duas novas bibliotecas essenciais:
- **`matplotlib.pyplot` (apelido `plt`):** A biblioteca fundamental para criar gráficos em Python. Ela nos dá controle total sobre a aparência das nossas visualizações.
- **`seaborn` (apelido `sns`):** Construída sobre o Matplotlib, ela nos permite criar gráficos estatísticos mais bonitos e complexos com menos código.

Vamos também definir um "tema" para o Seaborn, para que nossos gráficos já comecem com um visual mais profissional.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define o estilo visual dos gráficos
sns.set_theme(style="whitegrid")

### Passo 1: Preparando os Dados (Recapitulando o Módulo 2)

Para garantir que este notebook funcione de forma independente, vamos executar rapidamente todos os passos de limpeza que aprendemos no Módulo 2. Isso garante que estamos partindo de uma base de dados limpa e pronta para a análise.

In [ ]:
# 1. Carregar os dados
url = 'https://raw.githubusercontent.com/Miriam1s/mini_curso_inobar/main/vendas_lanchonete.csv'
df = pd.read_csv(url)

# 2. Limpar e transformar os dados
df['Preco'] = df['Preco'].str.replace(',', '.').astype(float)
df['ID_Cliente'].fillna(0, inplace=True)
df['ID_Cliente'] = df['ID_Cliente'].astype(int)
df['Data'] = pd.to_datetime(df['Data'])

# 3. Criar a coluna 'Total_Venda'
df['Total_Venda'] = df['Preco'] * df['Quantidade']

# 4. Verificar o resultado
print("Dados limpos e prontos para análise:")
df.info()
print("\nPrimeiras linhas do DataFrame final:")
df.head()

### Passo 2: O Poder do Agrupamento com `.groupby()`

Para responder nossas perguntas, precisamos agregar nossos dados. Por exemplo, para saber o total de vendas por produto, precisamos somar todas as vendas de "Coxinha", todas as de "Pastel", e assim por diante.

É aqui que entra o método `.groupby()`, uma das ferramentas mais poderosas do Pandas.

> **Analogia:** Pense no `.groupby()` como separar uma pilha de roupas para lavar. Você tem uma pilha de roupas misturadas (seu DataFrame). `df.groupby('Produto')` é o ato de criar montes separados: um para coxinhas, um para pastéis, etc. Depois de separar, você pode aplicar uma operação a cada monte, como somar (`.sum()`), contar (`.count()`) ou calcular a média (`.mean()`).

#### Pergunta 1: Qual produto gera mais receita?

**Lógica:**
1.  **Agrupar** os dados por `Produto`.
2.  Selecionar a coluna que queremos agregar: `Total_Venda`.
3.  Aplicar a operação de **soma** (`.sum()`).
4.  Ordenar os resultados para ver os melhores primeiro (`.sort_values()`).

In [ ]:
faturamento_por_produto = df.groupby('Produto')['Total_Venda'].sum().sort_values(ascending=False)

print("Faturamento total por produto:")
print(faturamento_por_produto)

#### Visualizando a Resposta

Uma tabela de números é boa, mas um gráfico é muito mais eficaz para comunicar esse resultado em uma apresentação.

Como estamos comparando categorias (produtos), um **gráfico de barras** é a escolha ideal.

In [ ]:
# Define o tamanho da figura (largura, altura)
plt.figure(figsize=(10, 6))

# Cria o gráfico de barras
sns.barplot(x=faturamento_por_produto.index, y=faturamento_por_produto.values, palette='viridis')

# Adiciona Título e Rótulos
plt.title('Faturamento Total por Produto', fontsize=16)
plt.xlabel('Produto', fontsize=12)
plt.ylabel('Faturamento Total (R$)', fontsize=12)

# Rotaciona os rótulos do eixo x para melhor leitura, se necessário
plt.xticks(rotation=45)

# Mostra o gráfico
plt.show()

> 💡 **Dica de Storytelling com Dados:**
> * **Sempre dê um título claro ao seu gráfico.** A audiência deve entender o que está vendo sem precisar de explicações.
> * **Rotule seus eixos.** O que significa o eixo X? E o Y? Inclua a unidade de medida (R$, %, etc.).
> * **Ordene suas barras.** Barras ordenadas (do maior para o menor, por exemplo) são muito mais fáceis de ler do que barras desordenadas.

**Insight:** A Coxinha é claramente nosso carro-chefe em faturamento!

#### Pergunta 2: Em qual dia da semana vendemos mais?

**Lógica:**
1.  Primeiro, precisamos criar uma coluna com o dia da semana. O Pandas torna isso fácil com o acessor `.dt` em colunas de data.
2.  Depois, agrupamos por essa nova coluna (`Dia_Semana`) e somamos o `Total_Venda`.
3.  **Pulo do Gato:** Os dias da semana virão em ordem alfabética. Precisamos reordená-los para a ordem correta (Segunda, Terça, etc.) para que o gráfico faça sentido.

In [ ]:
# 1. Criar a coluna com o nome do dia da semana
# Usamos .dt.day_name() para obter o nome do dia em inglês
df['Dia_Semana'] = df['Data'].dt.day_name()

# 2. Agrupar e somar
vendas_por_dia = df.groupby('Dia_Semana')['Total_Venda'].sum()

# 3. Corrigir a ordem dos dias
dias_ordem = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
vendas_por_dia = vendas_por_dia.reindex(dias_ordem).dropna()

print("Faturamento por dia da semana:")
print(vendas_por_dia)

#### Visualizando a Resposta

Novamente, um gráfico de barras é perfeito para comparar o desempenho de cada dia.

In [ ]:
plt.figure(figsize=(12, 6))

sns.barplot(x=vendas_por_dia.index, y=vendas_por_dia.values, palette='plasma')

plt.title('Faturamento por Dia da Semana', fontsize=16)
plt.xlabel('Dia da Semana', fontsize=12)
plt.ylabel('Faturamento Total (R$)', fontsize=12)

plt.show()

**Insight:** Nossas vendas se concentram no início da semana, com um pico na Terça-feira. Isso pode indicar uma oportunidade para promoções no final de semana para atrair mais clientes.

### Desafio Final! 🏆

Agora é sua vez de ser o analista!

**Sua missão:** O gerente quer saber qual produto vende mais em **quantidade**, não em faturamento. 

1.  Calcule a **quantidade total de itens vendidos por produto**.
2.  Crie um gráfico de barras para visualizar o resultado. Não se esqueça de dar um bom título e rotular os eixos!

In [ ]:
# Escreva seu código aqui!
# Dica: A lógica é muito parecida com a da Pergunta 1, mas você vai somar a coluna 'Quantidade'.


#### Solução do Desafio

In [ ]:
# 1. Agrupar por 'Produto' e somar a 'Quantidade'
quantidade_por_produto = df.groupby('Produto')['Quantidade'].sum().sort_values(ascending=False)

# 2. Criar o gráfico
plt.figure(figsize=(10, 6))
sns.barplot(x=quantidade_por_produto.index, y=quantidade_por_produto.values, palette='cividis')
plt.title('Quantidade de Itens Vendidos por Produto', fontsize=16)
plt.xlabel('Produto', fontsize=12)
plt.ylabel('Quantidade Total Vendida', fontsize=12)
plt.show()

## Conclusão do Minicurso

**Parabéns!** 🎉

Você completou a jornada completa de um analista de dados:
1.  Recebeu dados brutos e bagunçados (`.csv`).
2.  Explorou, limpou e preparou os dados para análise (Módulo 2).
3.  Transformou os dados limpos em respostas para perguntas de negócio usando agregações (`.groupby()`).
4.  Comunicou suas descobertas de forma clara e visual com gráficos (`matplotlib` e `seaborn`).

Este é o ciclo fundamental da análise de dados. As ferramentas e técnicas que você aprendeu aqui são a base para resolver problemas muito mais complexos no mundo real.

Lembre-se que este é apenas o começo. Continue praticando, seja curioso e explore a seção **"Como Continuar sua Jornada em Dados"** no `README.md` do nosso repositório para encontrar livros, plataformas e dicas para construir seu portfólio.

**Obrigado por participar e boa sorte na sua jornada de dados!**